<a href="https://colab.research.google.com/github/Itsuki-Hamano123/practice-casualeffect/blob/master/PBC/Calculation_ATE_ATT_From_PBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/機械学習練習/統計的因果推論/因果推論実践用/PBC

/content/drive/My Drive/機械学習練習/統計的因果推論/因果推論実践用/PBC


In [4]:
ls

Calculation_ATE_ATT_From_PBC.ipynb  datasets/  memo.txt


In [0]:
import pandas as pd
import numpy as np

#原発性胆汁性胆管炎（Primary biliary cholangitis：PBC）に対してD-ペニシラミンを用いたA/Bテストの結果からD-ペニシラミンの（因果）効果測定を行う

手順

1.   DataPreprocessing
2.   （誤った方法で）A/Bテストを評価
3.   傾向スコアの導出
4.   IPWで因果効果ATEを求めA/Bテストを評価
5.   傾向スコアマッチングで因果効果ATTを求めA/Bテストを評価

*   介入変数（treatment）：drug（D-ペニシラミン）
*   アウトカム（outcome）：status（生存したか死亡したか）

**実験データのカラム説明**
* id       = case number
* futime   = number of days between registration and the earlier of death,transplantion, or study analysis time in July, 1986
* status   = **（アウトカム）**0=alive, 1=liver transplant, 2=dead
* drug     = **（介入変数）** 1= D-penicillamine, 2=placebo
* age      = age in days
* sex      = 0=male, 1=female
* ascites  = presence of ascites: 0=no 1=yes
* hepato   = presence of hepatomegaly 0=no 1=yes
* spiders  = presence of spiders 0=no 1=yes
* edema    = presence of edema 0=no edema and no diuretic therapy for edema; .5 = edema present without diuretics, or edema resolved by diuretics; 1 = edema despite diuretic therapy
* bili     = serum bilirubin in mg/dl
* chol     = serum cholesterol in mg/dl
* albumin  = albumin in gm/dl
* copper   = urine copper in ug/day
* alk_phos = alkaline phosphatase in U/liter
* sgot     = SGOT in U/ml
* trig     = triglicerides in mg/dl
* platelet = platelets per cubic ml/1000
* protime  = prothrombin time in seconds
* stage    = histologic stage of disease




In [6]:
DATASET_PATH = './datasets/analytical_data.csv'
EXPERIMENT_DATA_DF = pd.read_csv(DATASET_PATH)
TREATMENT_COLUM = 'drug'
OUTCOME_COLUM = 'status'
print("ExperimentDataSize:{data}".format(data=EXPERIMENT_DATA_DF.shape))

ExperimentDataSize:(418, 20)


In [7]:
EXPERIMENT_DATA_DF

,id,futime,status,drug,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk_phos,sgot,trig,platelet,protime,stage
0,1,400,2,1,21464,1,1,1,1,1.0,14.5,261,2.60,156,1718,137.95,172,190,12.2,4
1,2,4500,0,1,20617,1,0,1,1,0.0,1.1,302,4.14,54,7394.8,113.52,88,221,10.6,3
2,3,1012,2,1,25594,0,0,0,0,0.5,1.4,176,3.48,210,516,96.1,55,151,12,4
3,4,1925,2,1,19994,1,0,1,1,0.5,1.8,244,2.54,64,6121.8,60.63,92,183,10.3,4
4,5,1504,1,2,13918,1,0,1,1,0.0,3.4,279,3.53,143,671,113.15,72,136,10.9,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,414,681,2,.,24472,1,.,.,.,0.0,1.2,.,2.96,.,.,.,.,174,10.9,3
414,415,1103,0,.,14245,1,.,.,.,0.0,0.9,.,3.83,.,.,.,.,180,11.2,4
415,416,1055,0,.,20819,1,.,.,.,0.0,1.6,.,3.42,.,.,.,.,143,9.9,3
416,417,691,0,.,21185,1,.,.,.,0.0,0.8,.,3.75,.,.,.,.,269,10.4,3


#①DataPreprocessing

今回はATE求める練習のため前処理は簡単に以下処理のみ行う
*   D-penicillamineが投与された場合は1、placeboの場合は0に値変更
*   欠測値を含むユーザを除く
*   肝移植したユーザ（status=1）をのぞく




In [0]:
def rewrite_categorical_binary_variable_value(data, stored_colum, flag_var, non_flag_var):
    """二値カテゴリ変数の書き換え

      WARNING:二値変数以外のカラムを渡して実行してはいけない

      Args:
         flag_val{category_name, current_val}(dict): 値を1にしたいカテゴリ {カテゴリ名, 現在格納されている値}
         non_flag_val{category_name, current_val}(dict): 値を0にしたいカテゴリ {カテゴリ名, 現在格納されている値}
    """
    data[stored_colum] = data[stored_colum].replace(flag_var['current_val'], flag_var['category_name']).replace(non_flag_var['current_val'], non_flag_var['category_name'])
    data[stored_colum] = data[stored_colum].replace(flag_var['category_name'], 1).replace(non_flag_var['category_name'], 0)
    return data


def remove_missing_value_line(data):
    """欠測値のある行を取り除く

     「.」のみまたは空白のデータが含まれる行を取り除く

    """
    # TODO query内のカラム指定をMySQLみたいに"*"にしたい
    preproccesing_data = data.query("not drug in ['.', '']")
    return preproccesing_data


def remove_status1_person(data):
    """肝移植したユーザを除く

     statusカラムが1となっている行を除く

    """
    preproccesing_data = data.query('not status == 1')
    return preproccesing_data

In [0]:
def df_change_type_to_double(data, expect_columns):
    print("function is df_change_type_to_double()")
    print(type(expect_columns),expect_columns)
    return "mock"

def preproccesing_data(origin_data):
    """データ前処理用

    Args:
        origin_data(df): 整形前のデータ

    Returns:
        preprocessed_data(df): 整形後のデータ
    """
    stored_colum, flag_var, non_flag_var = TREATMENT_COLUM, {'category_name' : 'D-penicillamine', 'current_val' : '1'}, {'category_name' : 'placebo', 'current_val' : '2'}
    preproccessed_data = rewrite_categorical_binary_variable_value(origin_data, stored_colum, flag_var, non_flag_var)
    
    preproccessed_data = remove_missing_value_line(preproccessed_data)
    
    preproccessed_data = remove_status1_person(preproccessed_data)

    df_change_type_to_double(preproccessed_data, preproccessed_data.columns)

    return preproccessed_data

In [14]:
PREPROCESSED_DATA = preproccesing_data(EXPERIMENT_DATA_DF)
print("PreprocessedDataSize:{data}".format(data=PREPROCESSED_DATA.shape))


function is df_change_type_to_double()
<class 'pandas.core.indexes.base.Index'> Index(['id', 'futime', 'status', 'drug', 'age', 'sex', 'ascites', 'hepato',
       'spiders', 'edema', 'bili', 'chol', 'albumin', 'copper', 'alk_phos',
       'sgot', 'trig', 'platelet', 'protime', 'stage'],
      dtype='object')
PreprocessedDataSize:(293, 20)


In [15]:
PREPROCESSED_DATA

,id,futime,status,drug,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk_phos,sgot,trig,platelet,protime,stage
0,1,400,2,1,21464,1,1,1,1,1.0,14.5,261,2.60,156,1718,137.95,172,190,12.2,4
1,2,4500,0,1,20617,1,0,1,1,0.0,1.1,302,4.14,54,7394.8,113.52,88,221,10.6,3
2,3,1012,2,1,25594,0,0,0,0,0.5,1.4,176,3.48,210,516,96.1,55,151,12,4
3,4,1925,2,1,19994,1,0,1,1,0.5,1.8,244,2.54,64,6121.8,60.63,92,183,10.3,4
5,6,2503,2,0,24201,1,0,1,0,0.0,0.8,248,3.98,50,944,93,63,.,11,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,308,1153,0,1,22347,1,0,1,0,0.0,0.4,246,3.58,24,797,91,113,288,10.4,2
308,309,994,0,0,21294,1,0,0,0,0.0,0.4,260,2.75,41,1166,70,82,231,10.8,2
309,310,939,0,1,22767,1,0,0,0,0.0,1.7,434,3.35,39,1713,171,100,234,10.2,2
310,311,839,0,1,13879,1,0,0,0,0.0,2.0,247,3.16,69,1050,117,88,335,10.5,2


In [0]:
def extract_any_category_data(data, target_categorical_name, target_categorical_num):
    extracted_data = data[data[target_categorical_name] == target_categorical_num]
    return extracted_data

In [17]:
TREATMENTGROUP_DATA = extract_any_category_data(PREPROCESSED_DATA, TREATMENT_COLUM, 1)
CONTROLGROUP_DATA = extract_any_category_data(PREPROCESSED_DATA, TREATMENT_COLUM, 0)
print(TREATMENTGROUP_DATA)
print(CONTROLGROUP_DATA)

      id  futime  status drug    age  ...    sgot trig platelet protime  stage
0      1     400       2    1  21464  ...  137.95  172      190    12.2      4
1      2    4500       0    1  20617  ...  113.52   88      221    10.6      3
2      3    1012       2    1  25594  ...    96.1   55      151      12      4
3      4    1925       2    1  19994  ...   60.63   92      183    10.3      4
8      9    2400       2    1  15526  ...  144.15   88      251      11      2
..   ...     ...     ...  ...    ...  ...     ...  ...      ...     ...    ...
301  302    1271       0    1  13806  ...      52  104      268    10.6      2
303  304    1230       0    1  12979  ...      45   75      246    10.8      3
307  308    1153       0    1  22347  ...      91  113      288    10.4      2
309  310     939       0    1  22767  ...     171  100      234    10.2      2
310  311     839       0    1  13879  ...     117   88      335    10.5      2

[148 rows x 20 columns]
      id  futime  status dr

In [18]:
print("TreatmentGroupSize:{treated_size}".format(treated_size = TREATMENTGROUP_DATA.shape[0]))
print("ControlGroupSize:{control_size}".format(control_size=CONTROLGROUP_DATA.shape[0]))

TreatmentGroupSize:148
ControlGroupSize:145


In [19]:
pd.set_option('display.max_columns', 50)
print("TreatmentGroupStatistics:\n{treated_statistics}".format(treated_statistics = TREATMENTGROUP_DATA.describe()))
print("ControlGroupStatistics:\n{control_statistics}".format(control_statistics=CONTROLGROUP_DATA.describe()))

TreatmentGroupStatistics:
               id       futime      status           age         sex  \
count  148.000000   148.000000  148.000000    148.000000  148.000000   
mean   155.135135  2060.459459    0.878378  19045.256757    0.878378   
std     86.482718  1102.393545    0.995947   3970.497901    0.327958   
min      1.000000    41.000000    0.000000   9598.000000    0.000000   
25%     81.750000  1262.000000    0.000000  16279.000000    1.000000   
50%    151.500000  1928.500000    0.000000  19246.500000    1.000000   
75%    227.500000  2689.750000    2.000000  21898.750000    1.000000   
max    311.000000  4556.000000    2.000000  28650.000000    1.000000   

            edema        bili     albumin  
count  148.000000  148.000000  148.000000  
mean     0.121622    2.850676    3.514730  
std      0.283453    3.728918    0.445448  
min      0.000000    0.300000    2.100000  
25%      0.000000    0.800000    3.217500  
50%      0.000000    1.350000    3.565000  
75%      0.000000

#②（誤った方法で）A/Bテストを評価

In [0]:
def calc_naive_tau(data, treatment_colum, outcome_colum):
    """　単純なoutcomeの差を求める

     WARNING:誤った効果測定の方法
　　 計算方法：介入群のoutcomeの平均値 - 対照群のoutcomeの平均値
    """
    print(data[data[treatment_colum] == 0])
    print(data[data[treatment_colum] == 1])
    return "mock"

In [21]:
calc_naive_tau(PREPROCESSED_DATA, TREATMENT_COLUM, OUTCOME_COLUM)

      id  futime  status drug    age  sex ascites hepato spiders  edema  bili  \
5      6    2503       2    0  24201    1       0      1       0    0.0   0.8   
6      7    1832       0    0  20284    1       0      1       0    0.0   1.0   
7      8    2466       2    0  19379    1       0      0       0    0.0   0.3   
9     10      51       2    0  25772    1       1      0       1    1.0  12.6   
10    11    3762       2    0  19619    1       0      1       1    0.0   1.4   
..   ...     ...     ...  ...    ...  ...     ...    ...     ...    ...   ...   
304  305    1216       0    0  15730    1       0      1       1    0.0   2.9   
305  306    1216       0    0  20597    1       0      1       0    0.0   0.6   
306  307    1149       0    0  11167    1       0      0       0    0.0   0.8   
308  309     994       0    0  21294    1       0      0       0    0.0   0.4   
311  312     788       0    0  12109    1       0      0       1    0.0   6.4   

    chol  albumin copper al

'mock'